# 0. Choosing/downloading dataset
QIIME release from UNITE website, following text is copied from the unite website and is a good example of what we could write in the thesis:

*Three sets of QIIME files are released, corresponding to the SHs resulting from clustering at the 3% distance (97% similarity) and 1% distance (99% similarity) threshold levels. The third set of files is the result of a dynamic use of clustering thresholds, such that some SHs are delimited at the 3% distance level, some at the 2.5% distance level, some at the 2% distance level, and so on; these choices were made manually by experts of those particular lineages of fungi. The syntax is the same throughout the three sets of files.*

*Each SH is given a stable name of the accession number type, here shown in the FASTA file of the dynamic set:*

\>SH099456.05FU_FJ357315_refs 
CACAATATGAAGGCGGGCTGGCACTCCTTGAGAGGACCGGC…

*SH099456 = accession number of the SH 05FU = global key release 5, organism group FUngi FJ357315 = GenBank/UNITE accession number of sequence chosen to represent the SH refs = this is a manually designated RefS (reps = this is an automatically chosen RepS*)

*In the corresponding text file, the classification string of the SH is found:*

*SH099456.05FU_FJ357315_refs k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Embellisia;s__Embellisia_planifunda*

*This specifies the hierarchical classification of the sequence. k = kingdom; p = phylum ; c = class ; o = order ; f = family ; g = genus ; and s = species. Missing information is indicated as "unidentified" item; “f__unidentified;” means that no family name for the sequence exists.*

- something more about which version (most recent one) and why
- something about the contents of the download (so 3 sets of QIIME release, all having 1 fasta file and 1 txt file, kind of like the explanation above)
- maybe some more explanation about what each file (fasta and txt) file contains, kind of like the explanation above but then in a full text.

# 1. Saving each QIIME release set into csv file
- Because we find csv files are easier to handle than the txt + fasta seperately
- CSV file are easy to manipulate and visualise with pandas
- working with the 3 releases (99%, 97% and dynamic thresholds) because idk honestly, I'm just not sure which one to use yet so I'll convert all of them:) if we end op finding a good reason to choose one or the other, we can still delete them

**CSV file contents**

TaxonomyXX.csv is a csv file with all info from the fasta and txt file from QIIME release + length of sequences

It has following columns: ['ACCNUM' 'KINGDOM' 'PHYLUM' 'CLASS' 'ORDER' 'FAMILY' 'GENUS' 'SPECIES', 'SEQ' 'LENGTH']

Function was made where the following steps were taken (basically what the code below does):
1. loading TXT file with pandas read_csv
2. Removing prefixes from each column (k__, p__, ...)
3. Entries with XX_XX_Incertae_sedis replaced with NaN for easier filtering with pandas as we will delete them anyway
4. Create list of sequences from FASTA file and add them to the dataframe (note: txt has same order as fasta file so we don't need to worry about losing the order later as the order is contained when adding the list to the dataframe)
5. Adding column 'SEQ' with sequences to dataframe
6. Adding column 'LENGTH' with lengths of sequences: for easy visualisation later on, length can be determined quickly with the BioPython/BioSeq module (maybe some short intermezzo about BioPython can be added here?)
7. saving pandas dataframe into csv with save_csv

Same steps for each QIIME set!! So 3 times, code below only shows it for 1 file but I'll put it in some fancy function so we can just do it as 'QIIME_to_csv(file.txt, file.fasta)' so it's prettier:) So code below isn't the final version but the steps taken won't change




In [ ]:
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt
import squarify

In [ ]:
# Loading Taxonomy data into dataframe, make column names/header
header = ['ACCNUM', 'KINGDOM', 'PHYLUM', 'CLASS', 'ORDER', 'FAMILY', 'GENUS', 'SPECIES']
DataframeAll = pd.read_csv("sh_taxonomy_qiime_ver9_99_29.11.2022.txt",
                           delimiter=r'[\t;]',
                           engine='python',
                           names=header)

# Removing prefixes from each column (k__, p__, ...)
for column in DataframeAll:
    DataframeAll[column] = DataframeAll[column].str.lstrip(f'{str(column)[0].lower()}__')
    if str(column) == 'SPECIES':
        DataframeAll[column] = DataframeAll[column].str.rstrip('_sp')

# Replacing XX_XX_Incertae_sedis with NaN
for column in DataframeAll:
    DataframeAll.loc[DataframeAll[str(column)].str.contains('sedis'), str(column)] = 'NaN'

# Create list of sequences
# Note: Fasta file has same order as txt file
sequences = []
for record in SeqIO.parse('sh_refs_qiime_ver9_99_29.11.2022.fasta', 'fasta'):
    sequences.append(record.seq)

# Adding column 'SEQ' with sequences to dataframe
# Adding column 'LENGTH' with lengths of sequences
# Note: format of sequences = BioSeq
DataframeAll['SEQ'] = sequences
DataframeAll['LENGTH'] = DataframeAll['SEQ'].str.len()

# Saving csv file for further use
DataframeAll.to_csv('Taxonomy99.csv', index=None)

# 2. Data Filtering Pre-Analysis
Filtering was done following these steps:
1. check database for duplicates (these were printed to manually double check they were the same) and delete one of the duplicate duos so no more duplicates in the dataset
2. Check for rows with NaN in dataset at any of the taxonomic levels and delete these rows, we need labelled data for training

Save into csv for further use

In [ ]:
df = pd.read_csv("Taxonomy99.csv")
# Check database for duplicates
print(df[df.duplicated(subset='SEQ', keep=False)])
'''
                              ACCNUM  ... LENGTH
26779   SH2065224.09FU_KX065947_reps  ...    484
59005   SH2065231.09FU_JN684766_refs  ...    484
80104   SH1472029.09FU_MZ322935_reps  ...    471
158833  SH1472048.09FU_JX272968_refs  ...    471

[4 rows x 10 columns]
'''
# Delete one of the 2 of the duplicates
df_noduplicates = df.drop_duplicates(subset='SEQ')
print(df.shape)
print(df_noduplicates.shape)
'''
(197557, 10)
(197555, 10)
'''
# Check for rows with NaN in one of the columns
print(df_noduplicates[df_noduplicates.isna().any(axis=1)])
'''
                                 ACCNUM  ... LENGTH
4       SH1367668.09FU_UDB03376623_reps  ...    538
5        SH1380440.09FU_UDB0684437_reps  ...    519
6       SH2027613.09FU_UDB02118143_reps  ...    583
9          SH1448937.09FU_MH178258_reps  ...    515
10      SH1367682.09FU_UDB03384410_reps  ...    538
...                                 ...  ...    ...
197552   SH2003702.09FU_UDB0193281_reps  ...    609
197553  SH2003697.09FU_UDB02276242_reps  ...    603
197554  SH2003700.09FU_UDB02257150_reps  ...    611
197555   SH2003704.09FU_UDB0283399_reps  ...    604
197556  SH2003723.09FU_UDB02262136_reps  ...    605

[93901 rows x 10 columns]
'''
# Drop all rows with NaN in one of the columns
df_final = df_noduplicates.dropna()
print(df_final)
'''
                                 ACCNUM  ... LENGTH
0       SH2007016.09FU_UDB03582617_reps  ...    600
1        SH1902468.09FU_UDB0227636_reps  ...    591
2       SH1949090.09FU_UDB02678888_reps  ...    569
3       SH1857229.09FU_UDB06071185_reps  ...    625
7          SH2007337.09FU_AY340038_reps  ...    618
...                                 ...  ...    ...
197538  SH2003910.09FU_UDB03776145_reps  ...    559
197539  SH2003916.09FU_UDB01950582_reps  ...    552
197540     SH2003923.09FU_JQ666517_reps  ...    546
197541  SH2003985.09FU_UDB03630955_reps  ...    541
197546  SH2003333.09FU_UDB03237010_reps  ...    629

[103654 rows x 10 columns]
'''
# Plot sizes of cleaning of dataframe (kinda stupid with the 2 duplicates?)
df_sizes = pd.DataFrame({'Size':[103654,93901,2], 'Consisting of':["Final Dataset", "Entries with NaN", "Duplicates"]})
squarify.plot(sizes=df_sizes['Size'], label=df_sizes['Consisting of'], alpha=.8)
plt.axis('off')
plt.savefig('Sizes.png')
'''
Sizes.png saved in folder for idk cause it looks terrible
'''
# Save df_final to csv for further use and analysis
df_final.to_csv('Taxonomy99Final.csv', index=None)

# DONE => analysis can start

# Data preparation after analysis (don't write this yet)
Preparation of the data that hopefully follow out of the data analysis.

*Just dropping them here because I'm going to forget them otherwise and I haven't made a seperate file yet for filtering after analysis. But these are some open questions that we should maybe ask prof or abiodun.*

1. Should we maybe check if sequences containing a certain % of ambigous nucleotides are available? (a certain % of ambigous nucleotides could be due to sequencing errors and should be avoided) and delete these with higher %
2. maybe delete ambigous nucleotides all together from sequences as it will simplify the kmer- and One-Hot-Encoding A LOT (ATGC instead of ATCGNFUCKTHIS). If we do, we should do it after/during the analysis and check the length distributions/mean/whatever and hopefully conclude it doesn't really change much so we can just delete them
3. Cut the shorter and longer sequences because for OHE, we will need to padd them with additional N, as the neural network expects all same lengths. And we want same training/validation/test set for both OHE and kmers. Pretty distribution histogram (+literatures) will help us prove this point hopefully
4. Maybe delete 'rare' examples (such as only 1 entry for a certain phylum), idk? Let's check how many there are for each taxonomic level and write some stuff on why these are not useful for training neural networks
5. Maybe instead of deleting these 'rare' examples, we could augment the data (complement, reverse, reverse complement) to have more entries for the 'rare' examples